In [1]:
import os, json, time, datetime as dt, csv, pathlib
import sys
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
sys.path.append(os.path.abspath(".."))

from src.utils import validate_df, safe_filename

DATA_RAW = pathlib.Path("data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
Stock = os.getenv("STOCK")
print("Loaded ALPHAVANTAGE_API_KEY?", bool(ALPHA_KEY))

Loaded ALPHAVANTAGE_API_KEY? True


In [2]:
SYMBOL = Stock
use_alpha = bool(ALPHA_KEY)
print("Using Alpha Vantage:", use_alpha)

if use_alpha:
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": SYMBOL,
        "outputsize": "compact",
        "apikey": ALPHA_KEY,
        "datatype": "json"
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js.keys() if "Time Series" in k]
    assert key, f"Unexpected response keys: {list(js.keys())}"
    series = js[key[0]]
    df_api = (pd.DataFrame(series).T
              .rename_axis('date')
              .reset_index())
    # keep a couple columns and coerce types
    df_api = df_api[['date', '5. adjusted close']].rename(columns={'5. adjusted close': 'adj_close'})
    df_api['date'] = pd.to_datetime(df_api['date'])
    df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period="6mo", interval="1d").reset_index()[['Date','Close']]
    df_api.columns = ['date','adj_close']

df_api = df_api.sort_values('date').reset_index(drop=True)
msgs = validate_df(df_api, required_cols=['date','adj_close'], dtypes_map={'date':'datetime64[ns]','adj_close':'float'})
print(msgs)

fname = safe_filename(prefix="api", meta={"source": "alpha" if use_alpha else "yfinance", "symbol": SYMBOL})
out_path = DATA_RAW / fname
df_api.to_csv(out_path, index=False)
print("Saved:", out_path)

Using Alpha Vantage: True


AssertionError: Unexpected response keys: ['Information']

In [ ]:
FUNCTION = "TIME_SERIES_DAILY_ADJUSTED"
OUTPUT_SIZE = "compact"
SCRAPE_URL = (
    f"https://www.alphavantage.co/query"
    f"?function={FUNCTION}&symbol={SYMBOK}&outputsize={OUTPUT_SIZE}&apikey={ALPHA_KEY}"
)
headers = {"User-Agent": "AFE-Course-Notebook/1.0 (contact: fj2034@nyu.edu)"}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table')
    rows = []
    for tr in table.find_all('tr'):
        cells = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
        if cells:
            rows.append(cells)
    # assume first row is header
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print("Scrape failed (demoing with inline HTML).", e)
    html = """
    <table>
      <tr><th>Ticker</th><th>Price</th></tr>
      <tr><td>AAA</td><td>101.2</td></tr>
      <tr><td>BBB</td><td>98.7</td></tr>
    </table>
    """
    soup = BeautifulSoup(html, 'html.parser')
    rows = []
    for tr in soup.find_all('tr'):
        cells = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
        if cells:
            rows.append(cells)
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')

msgs2 = validate_df(df_scrape, required_cols=list(df_scrape.columns), dtypes_map={})
print(msgs2)

fname2 = safe_filename(prefix="scrape", meta={"site": "example", "table": "markets"})
out_path2 = DATA_RAW / fname2
df_scrape.to_csv(out_path2, index=False)
print("Saved:", out_path2)